## Build the Neural Network

In [1]:
## importing libraries
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

- `torch.nn` - provides all the building blocks you need to build your own neural network.
- Every module is a subclass of `nn.Module`.
- A neural network module will comprise of layer modules

In [5]:
## Check GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


### Define the Class

In [4]:
class NeuralNetwork(nn.Module): ## inherits nn.Module
    def __init__(self):  ## initialize Neural Network
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):  ## all operations on input data are defined here
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
## Creating a instance of Neural Netwok module and moving it to GPU
model = NeuralNetwork().to(device) 
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
## Creating a input
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
print("Logits :", logits)

## getting probabilities from sores
pred_probab = nn.Softmax(dim=1)(logits)
print("\nClass Probabilities :", pred_probab)

## predicted class
y_pred = pred_probab.argmax(1)
print(f"\nPredicted class: {y_pred}")

Logits : tensor([[-0.0127, -0.0576, -0.0204, -0.0743,  0.0919, -0.0637,  0.0417,  0.0070,
         -0.0681,  0.0135]], grad_fn=<AddmmBackward0>)

Class Probabilities : tensor([[0.1000, 0.0956, 0.0993, 0.0940, 0.1111, 0.0951, 0.1056, 0.1020, 0.0946,
         0.1027]], grad_fn=<SoftmaxBackward0>)

Predicted class: tensor([4])


### Model Layers

In [9]:
## dummy input
input_image = torch.rand(3,28,28) # 28x28 images of batch size 3
print("Input: ", input_image.size())

Input:  torch.Size([3, 28, 28])


> nn.Flatten  

- This layer flattens the images. Here, it will convert each 2D 28x28 image into a contiguous array of 784 pixel values ( the minibatch dimension (at dim=0) is maintained).

In [10]:
flatten = nn.Flatten() ## flatten at dim=0
flat_image = flatten(input_image)
print('Input after flattening operation:', flat_image.size())

Input after flattening operation: torch.Size([3, 784])


> nn.Linear

-  This layer applies a linear transformation on the input using its stored weights and biases.

In [11]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print("Linear layer effect: ", hidden1.size())

Linear layer effect:  torch.Size([3, 20])


> nn.ReLU

- ReLU activation function

In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0765, -0.2275, -0.2063, -0.0518,  0.2393,  0.2751,  0.0668,  0.4275,
          0.1290, -0.1931,  0.0763, -0.1507,  0.2638,  0.0401, -0.4388, -0.5212,
          0.3772, -0.3718,  0.3522, -0.4703],
        [ 0.2904, -0.0241, -0.2105, -0.0668,  0.1269,  0.6301,  0.2063,  0.1487,
          0.1072,  0.2201,  0.3875, -0.6165,  0.1536,  0.0438, -0.2857, -0.1727,
          0.1878, -0.1233,  0.1325,  0.0459],
        [ 0.5166, -0.3252,  0.0307, -0.3154,  0.5411,  0.3619,  0.0159,  0.1711,
          0.1616,  0.2657,  0.4050, -0.0355,  0.1383,  0.1713, -0.5681, -0.3367,
          0.1887, -0.2423,  0.0710, -0.3843]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0765, 0.0000, 0.0000, 0.0000, 0.2393, 0.2751, 0.0668, 0.4275, 0.1290,
         0.0000, 0.0763, 0.0000, 0.2638, 0.0401, 0.0000, 0.0000, 0.3772, 0.0000,
         0.3522, 0.0000],
        [0.2904, 0.0000, 0.0000, 0.0000, 0.1269, 0.6301, 0.2063, 0.1487, 0.1072,
         0.2201, 0.3875, 0.0000, 0.1536, 0.0438, 0.00

> nn.Sequential

- It is an ordered container of modules(layers). Input will pass through the modules in same order as they are defined.

In [13]:
## creating module contaner
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(logits.size())

torch.Size([3, 10])


> nn.Softmax

- Softmax activation (for getting class probabilities)

In [14]:
softmax = nn.Softmax(dim=1)  ## dim=1 indicates not to compute softmax across inputs
pred_probab = softmax(logits)
print('predicted probabilities : ', pred_probab)

predicted probabilities :  tensor([[0.0999, 0.1050, 0.0890, 0.1034, 0.0858, 0.1013, 0.1146, 0.1038, 0.1206,
         0.0767],
        [0.0965, 0.1029, 0.0939, 0.1015, 0.0937, 0.1094, 0.1055, 0.1009, 0.1144,
         0.0812],
        [0.0870, 0.1034, 0.0979, 0.1144, 0.0874, 0.1093, 0.1126, 0.0946, 0.1124,
         0.0811]], grad_fn=<SoftmaxBackward0>)


### Model Parameters

- `parameters()` and `named_parameters()` give accessed to all the learnable parameters of every layer inside a model, kept tracked by `nn.module`.

In [15]:
print(f"Model structure: {model}\n\n")


Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)




In [16]:
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0028, -0.0293, -0.0289,  ..., -0.0024, -0.0004, -0.0161],
        [-0.0229, -0.0284,  0.0346,  ..., -0.0306,  0.0326, -0.0112]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0287, -0.0113], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0272,  0.0141,  0.0198,  ..., -0.0073, -0.0225, -0.0143],
        [ 0.0216,  0.0424,  0.0432,  ...,  0.0342, -0.0335, -0.0216]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Size: torch.Size([512]) | Values : tensor([-0.0219,  0.0219], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.4.weight | Size: torch.Size([10, 512]) | Values : tensor([[ 0.0434, -0.0040, -0.0349,  ..., -0.0346,  0.0104,  0.0094],
        [ 0.0289, -0.0419,  0.0259,  ..., -0.0230, -0.0222,  0.0167]],
       grad_fn=<SliceBackward0>)